Hazırlanmış Dataframeyi çek

In [1]:
import pandas as pd
df = pd.read_csv("dfNew.csv")
df

,Unnamed: 0,Texts,Type
0,0,yıldır kullanıyorum memnunum,0
1,1,yıldır kullanıyorum müthiş,0
2,2,ürün bugün elime geçti çok fazla inceleme fırs...,0
3,3,almaya karar verdim hemencecik geldi keyifle k...,0
4,4,günlük kullanımınızı çok çok iyi karsılıyor kı...,0
...,...,...,...
2072,2072,ballı rus pastası tarifi,2
2073,2073,pelmeni tarifi,2
2074,2074,begova çorbası tarifi,2
2075,2075,pita ekmeği tarifi,2


In [2]:
from nltk.tokenize import word_tokenize
tokenized = [ word_tokenize(i) for i in df["Texts"]]

### QUESTION 1

Word2Vec

In [3]:
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
Word2VecNOstem = Word2Vec(tokenized, min_count=3)

In [4]:
Word2VecNOstem.wv.most_similar(positive=["güzel","kullanışlı"],negative=["kargo"])

[('değil', 0.998813807964325),
 ('çok', 0.9987935423851013),
 ('uygun', 0.9987845420837402),
 ('daha', 0.9987837076187134),
 ('olması', 0.9987736344337463),
 ('bir', 0.9987677931785583),
 ('olarak', 0.9987658858299255),
 ('oldukça', 0.9987534880638123),
 ('gibi', 0.9987496137619019),
 ('göre', 0.9987471699714661)]

Türkçe Stemming

In [5]:
from snowballstemmer import TurkishStemmer
turkStem=TurkishStemmer()
StemmedWords = []
StemmedSentences = tokenized
count = 0

for sentence in tokenized:
    for word in sentence:
        StemmedWords.append(turkStem.stemWord(word))
    for i in range(len(StemmedWords)):
        StemmedSentences[count][i] = StemmedWords[i]
    
    StemmedWords = []
    count = count + 1

count = 0

In [6]:
Word2VecYESstem = Word2Vec(StemmedSentences, min_count=3)
Word2VecYESstem.wv.most_similar(positive=["geliyor","gel"],negative=["git"])

[('gerçek', 0.9929010272026062),
 ('hepsiburadaco', 0.992763102054596),
 ('al', 0.9927240610122681),
 ('zama', 0.9926722049713135),
 ('ev', 0.9926694631576538),
 ('tavsi', 0.9926194548606873),
 ('be', 0.992576539516449),
 ('uzak', 0.9925687909126282),
 ('ba', 0.9925214052200317),
 ('ke', 0.9925079345703125)]

In [7]:
#import numpy as np
#print(np.linalg.norm(Word2VecNOstem["iyi"] - Word2VecNOstem["geldi"]))

FastText

In [8]:
from gensim.models.fasttext import FastText

In [9]:
ft_model = FastText(tokenized, min_count=3)

In [10]:
#Semantically similar words
ft_model.wv.most_similar(positive=["güzel","kullanışlı"],negative=["kargo"])

[('kullanış', 0.9999741315841675),
 ('kullan', 0.9999726414680481),
 ('kullanım', 0.9999719858169556),
 ('kullanır', 0.9999715089797974),
 ('kullanıla', 0.9999709129333496),
 ('kullanan', 0.9999707937240601),
 ('kullanıslı', 0.9999707937240601),
 ('kullana', 0.9999706149101257),
 ('kullanıl', 0.999970555305481),
 ('kullandı', 0.9999700784683228)]

TF IDF

In [11]:
from gensim.models.tfidfmodel import TfidfModel

In [12]:
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim import corpora

In [13]:
mydict = corpora.Dictionary([simple_preprocess(line) for line in df["Texts"]])
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in df["Texts"]]
Tfidf_model = TfidfModel(corpus)

Bag of Words (Vectorization)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [15]:
trstopwords = ["acaba","ama","ancak","artık","asla","aslında","az","bana","bazen","bazı","bazıları","bazısı","belki","ben","beni","benim","beş","bile","bir","birçoğu","birçok","birçokları","biri","birisi","birkaç","birkaçı","birşey","birşeyi","biz","bize","bizi","bizim","böyle","böylece","bu","buna","bunda","bundan","bunu","bunun","burada","bütün","çoğu","çoğuna","çoğunu","çok","çünkü","da","daha","de","değil","demek","diğer","diğeri","diğerleri","diye","dolayı","elbette","en","fakat","falan","felan","filan","gene","gibi","hangi","hangisi","hani","hatta","hem","henüz","hep","hepsi","hepsine","hepsini","her","her biri","herkes","herkese","herkesi","hiç","hiç kimse","hiçbiri","hiçbirine","hiçbirini","için","içinde","ile","ise","işte","kaç","kadar","kendi","kendine","kendini","ki","kim","kime","kimi","kimin","kimisi","madem","mı","mi","mu","mü","nasıl","ne","ne kadar","ne zaman","neden","nedir","nerde","nerede","nereden","nereye","nesi","neyse","niçin","niye","ona","ondan","onlar","onlara","onlardan","onların","onu","onun","orada","oysa","oysaki","öbürü","ön","önce","ötürü","öyle","sana","sen","senden","seni","senin","siz","sizden","size","sizi","sizin","son","sonra","seobilog","şayet","şey","şimdi","şöyle","şu","şuna","şunda","şundan","şunlar","şunu","şunun","tabi","tamam","tüm","tümü","üzere","var","ve","veya","veyahut","ya","ya da","yani","yerine","yine","yoksa","zaten","zira","kimse","zaman"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.Texts, df.Type, test_size=0.25)

In [17]:
cv = CountVectorizer(stop_words = frozenset(trstopwords))
X_train_BoW = cv.fit_transform(X_train.values)
X_test_BoW = cv.transform(X_test)

TF IDF

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf_cv = TfidfVectorizer(stop_words = frozenset(trstopwords))
X_train_Tfidf = tfidf_cv.fit_transform(X_train.values)
X_test_Tfidf = tfidf_cv.transform(X_test)

### QUESTION 2

KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn = KNeighborsClassifier()

knn_model = knn.fit(X_train_BoW,y_train)
accuracy1 = knn_model.score(X_test_BoW,y_test)

pred1 = knn_model.predict(X_test_BoW)
result1 = pd.DataFrame(classification_report(y_test,pred1,output_dict=True))

Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
dc = DecisionTreeClassifier()

dc_model = dc.fit(X_train_BoW,y_train)
accuracy2 = dc_model.score(X_test_BoW,y_test)

pred2 = dc_model.predict(X_test_BoW)
result2 = pd.DataFrame(classification_report(y_test,pred2,output_dict=True))

Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf = RandomForestClassifier()

rf_model = rf.fit(X_train_BoW,y_train)
accuracy3 = rf_model.score(X_test_BoW,y_test)

pred3 = rf_model.predict(X_test_BoW)
result3 = pd.DataFrame(classification_report(y_test,pred3,output_dict=True))

Results into Table

In [26]:
#PUTTING THE RESULTS INTO A TABLE
#pip install tabulate
from tabulate import tabulate

Thedata = [
    [accuracy1, result1["accuracy"]["precision"], result1["accuracy"]["recall"], result1["accuracy"]["f1-score"]],
    [accuracy2, result2["accuracy"]["precision"], result2["accuracy"]["recall"], result2["accuracy"]["f1-score"]],
    [accuracy3, result3["accuracy"]["precision"], result3["accuracy"]["recall"], result3["accuracy"]["f1-score"]]]

head = ["Accuracy","Precision","Recall","F1-Score"]


In [27]:
print(tabulate(Thedata, headers=head, tablefmt="grid"))

+------------+-------------+----------+------------+
|   Accuracy |   Precision |   Recall |   F1-Score |
+============+=============+==========+============+
|   0.842308 |    0.842308 | 0.842308 |   0.842308 |
+------------+-------------+----------+------------+
|   0.855769 |    0.855769 | 0.855769 |   0.855769 |
+------------+-------------+----------+------------+
|   0.857692 |    0.857692 | 0.857692 |   0.857692 |
+------------+-------------+----------+------------+


Doing same things with TF IDF vectorizer

KNN

In [28]:
knn_model = knn.fit(X_train_Tfidf,y_train)
accuracy1 = knn_model.score(X_test_Tfidf,y_test)

pred1 = knn_model.predict(X_test_Tfidf)
result1 = pd.DataFrame(classification_report(y_test,pred1,output_dict=True))

Decision Tree

In [29]:
dc_model = dc.fit(X_train_Tfidf,y_train)
accuracy2 = dc_model.score(X_test_Tfidf,y_test)

pred2 = dc_model.predict(X_test_Tfidf)
result2 = pd.DataFrame(classification_report(y_test,pred2,output_dict=True))

Random Forest

In [30]:
rf_model = rf.fit(X_train_Tfidf,y_train)
accuracy3 = rf_model.score(X_test_Tfidf,y_test)

pred3 = rf_model.predict(X_test_Tfidf)
result3 = pd.DataFrame(classification_report(y_test,pred3,output_dict=True))

Results to table

In [31]:
Thedata = [
    [accuracy1, result1["accuracy"]["precision"], result1["accuracy"]["recall"], result1["accuracy"]["f1-score"]],
    [accuracy2, result2["accuracy"]["precision"], result2["accuracy"]["recall"], result2["accuracy"]["f1-score"]],
    [accuracy3, result3["accuracy"]["precision"], result3["accuracy"]["recall"], result3["accuracy"]["f1-score"]]]

head = ["Accuracy","Precision","Recall","F1-Score"]


In [32]:
print(tabulate(Thedata, headers=head, tablefmt="grid"))

+------------+-------------+----------+------------+
|   Accuracy |   Precision |   Recall |   F1-Score |
+============+=============+==========+============+
|   0.838462 |    0.838462 | 0.838462 |   0.838462 |
+------------+-------------+----------+------------+
|   0.853846 |    0.853846 | 0.853846 |   0.853846 |
+------------+-------------+----------+------------+
|   0.857692 |    0.857692 | 0.857692 |   0.857692 |
+------------+-------------+----------+------------+
